<a href="https://colab.research.google.com/github/DataSavvyYT/experiments/blob/main/stock_prediction/tune_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Install Unsloth and dependencies (Fastest way for Colab)
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes
!pip install yfinance pandas scikit-learn

In [ ]:
import os
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import Dataset
import yfinance as yf
import pandas as pd

In [ ]:
# 2. Configuration
max_seq_length = 2048
dtype = None # Auto detection
load_in_4bit = True # 4-bit quantization to fit in free Colab GPU

In [ ]:
# 3. Load Llama 3 Model (Pre-quantized 4-bit)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
# 4. Create Sample Dataset using yfinance (AAPL History)
def create_stock_dataset(ticker="AAPL", period="2y", window_size=10):
    # Fetch data
    stock = yf.Ticker(ticker)
    df = stock.history(period=period)

    # Calculate simple features (rounded to 2 decimals for tokenization ease)
    closes = df['Close'].round(2).astype(str).tolist()

    data_points = []

    # Create sliding window sequences
    for i in range(len(closes) - window_size):
        past_prices = ", ".join(closes[i : i + window_size])
        target_price = closes[i + window_size]

        # Format as an instruction
        text = (
            f"Below is a history of stock prices. Predict the next closing price.\n\n"
            f"### Input:\n{past_prices}\n\n"
            f"### Response:\n{target_price}<|end_of_text|>"
        )
        data_points.append({"text": text})

    return Dataset.from_pandas(pd.DataFrame(data_points))


In [ ]:
# Generate dataset
dataset = create_stock_dataset()
print(f"Generated {len(dataset)} training samples. Example:\n{dataset[0]['text']}")


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # LoRA rank
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

In [ ]:
# 6. Train the Model
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Small step count for demo speed
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
trainer.train()

In [ ]:
# 7. Inference (Test Prediction)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Create a dummy input (last 10 days of prices)
test_input = "180.50, 181.20, 179.80, 182.00, 181.50, 183.10, 182.90, 184.00, 183.50, 185.00"
prompt = f"Below is a history of stock prices. Predict the next closing price.\n\n### Input:\n{test_input}\n\n### Response:\n"


In [ ]:
inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens = 10,
    use_cache = True,
    eos_token_id = tokenizer.eos_token_id
)


In [ ]:
prediction = tokenizer.batch_decode(outputs)
print(f"\n\nPREDICTION OUTPUT:\n{prediction[0].split('### Response:')[1]}")